In [ ]:
!pip install transformers

In [ ]:
!pip install datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

In [ ]:
from datasets import load_dataset
dataset = load_dataset("haritzpuerto/steam_product_descriptions_persuasive")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/645 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/9.03M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4360 [00:00<?, ? examples/s]

In [ ]:
def preprocess(example):
  input_text = f"Genres: {example['Genres']}, Categories: {example['Categories']}, About: {example['About the game']}"
  output_text = example['Persuasive description']
  model_inputs = tokenizer(input_text, max_length=512, truncation=True)
  labels = tokenizer(output_text, max_length=512, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

tok_dataset = dataset.map(preprocess,remove_columns=dataset["train"].column_names)

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

lora = LoraConfig(
  task_type=TaskType.SEQ_2_SEQ_LM,
  r=16,
  lora_alpha=32,
  lora_dropout=0.05,
  bias="none",
  target_modules=["q_proj", "v_proj"]
)

model = get_peft_model(model, lora)

In [ ]:
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments

collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

args = TrainingArguments(
  output_dir="./gamead-output",
  save_strategy="epoch",
  num_train_epochs=3,
  per_device_train_batch_size=4,
  per_device_eval_batch_size=4,
  logging_steps=50,
  fp16=True
  )

trainer = Trainer(
  model=model,
  data_collator=collator,
  args=args,
  tokenizer=tokenizer,
  train_dataset=tok_dataset["train"],
  eval_dataset=tok_dataset.get('validation')
)

trainer.train()

<ipython-input-8-2ec0e392e2e7>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abhinav-pattanshetti (abhinav-pattanshetti-pes-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,4.276400
100,3.852800
150,3.656300
200,3.537000
250,3.459300
300,3.380600
350,3.369900
400,3.281100
450,3.235800
500,3.244400


TrainOutput(global_step=3270, training_loss=3.027461402496431, metrics={'train_runtime': 638.4467, 'train_samples_per_second': 20.487, 'train_steps_per_second': 5.122, 'total_flos': 3767728338468864.0, 'train_loss': 3.027461402496431, 'epoch': 3.0})

In [ ]:
import torch

def generate_ad(genres, categories, about):
  input_text = f"Genres: {genres}, Categories: {categories}, About: {about}"
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
  output_ids = model.generate(input_ids=input_ids, max_length=512, num_beams=4, early_stopping=True)  # Changed line
  output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  return output

In [ ]:
ad1 = generate_ad(
    genres='RPG, Fantasy',
    categories = 'Adventure, Open World, Magic',
    about= "Embark on an epic journey through the vast and enchanting realm of Aerthos! As a chosen hero, you'll explore sprawling landscapes teeming with mythical creatures, ancient ruins, and hidden secrets. Master powerful magic, forge alliances with diverse companions, and unravel a compelling storyline that will determine the fate of the land. With hundreds of hours of gameplay, deep character customization, and a vibrant world to discover, Aerthos awaits your legend!"
)

ad2 = generate_ad(
    genres='Strategy, Simulation',
    categories='City Builder, Management, Economy',
    about="Become the ultimate architect and leader in Metropolis 2049! Design and construct a thriving futuristic city from the ground up, managing intricate resource networks, advanced technologies, and the diverse needs of your citizens. Navigate economic challenges, research groundbreaking innovations, and expand your metropolis into a shining beacon of the future. With deep simulation mechanics and endless possibilities for urban planning, can you build the city of tomorrow?"
)

ad3 = generate_ad(
    genres='Action, Indie',
    categories='Platformer, Metroidvania, Pixel Art',
    about="Leap into the pixelated world of Aethelgard and guide the agile warrior Anya on a perilous quest to reclaim her stolen homeland! Explore intricate interconnected levels filled with challenging platforming, hidden upgrades, and formidable foes. Discover new abilities, backtrack through the map to uncover secrets, and engage in fast-paced combat. With its retro charm and rewarding exploration, Aethelgard offers a nostalgic yet fresh Metroidvania experience."
)

print(ad1)
print('\n\n')
print(ad2)
print('\n\n')
print(ad3)

Embark on an epic journey through the vast and enchanting realm of Aerthos, where you will master powerful magic, forge alliances with diverse characters, and unravel a compelling storyline that will determine the fate of the land. This isn’t just a game; it’s a thrilling experience that will leave you with an unforgettable journey into the heart of the world. With hundreds of hours of gameplay, deep character customization, and a vibrant world to discover your destiny, you’ll be immersed in a world filled with awe and wonder.



Step into the role of the ultimate architect and leader in Metropolis 2049, where you can master the design of a futuristic city from the ground up, navigating intricate resource networks, advanced technologies, and the diverse needs of your citizens. Navigate economic challenges, research groundbreaking innovations, and transform your metropolis into a shining beacon of the future! With deep simulation mechanics and endless possibilities for urban planning, y